In [1]:
import pandas as pd
import numpy as np

In [2]:
usecols = ['identifier', 'shipper_party_name', 'shipper_party_address_1',
       'shipper_party_address_2', 'shipper_party_address_3',
       'shipper_party_address_4', 'city', 'state_province', 'zip_code',
       'country_code']

In [3]:
dtype = {'identifier':str,'shipper_party_name':str, 'shipper_party_address_1':str,
       'shipper_party_address_2':str, 'shipper_party_address_3':str,
       'shipper_party_address_4':str, 'city':'category', 'state_province':'category', 'zip_code':'category',
       'country_code':'category'}

In [4]:
data = pd.read_csv('raw_data/2018/AMSShippers-2018.csv',usecols=usecols,dtype=dtype)

### Simple string cleaning

In [ ]:
len(data['shipper_party_name'].unique())

In [ ]:
len(data['shipper_party_name'].str.replace(',','',regex=False).unique())

In [ ]:
len(data['shipper_party_name'].str.replace(',','',regex=False).str.replace('.','',regex=False).unique())

In [ ]:
#data['shipper_party_name'].value_counts().head()

In [ ]:
#data.loc[data['shipper_party_name'].str.contains('dhl',case=False,regex=False,na=False)]

### Match company by TFIDF
https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

In [ ]:
import re
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

company_names = pd.Series(data['shipper_party_name'].unique()).dropna()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names)

In [ ]:
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [ ]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

In [ ]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [ ]:
matches_df = get_matches_df(matches, company_names, top=100000)
matches_df.to_pickle('matches_df.pkl')

In [ ]:
pd.read_pickle('matches_df.pkl').to_csv('matches_df.csv')

### Match by Levenshtein distance

In [5]:
from Levenshtein import distance,ratio
from fuzzywuzzy import fuzz

In [6]:
def apply_distance(col1,col2):
    return distance(col1,col2)
apply_distance_vectorize = np.vectorize(apply_distance)
def apply_ratio(col1,col2):
    return ratio(col1,col2)
apply_ratio_vectorize = np.vectorize(apply_ratio)
def apply_fuzz_partial_ratio(col1,col2):
    return fuzz.partial_ratio(col1,col2)
apply_fuzz_partial_ratio_vectorize = np.vectorize(apply_fuzz_partial_ratio)

In [7]:
#shipper = pd.Series(data['shipper_party_name'].unique()).dropna()

In [8]:
shipper = pd.Series(data['shipper_party_name'].str.replace(',','',regex=False).str.replace('.','',regex=False).unique()).dropna()

In [ ]:
#apply_ratio_vectorize(shipper.values,shipper.values)

In [ ]:
#import itertools
#pd.DataFrame(list(itertools.product(*shipper.values)), columns=['left', 'right'])

In [ ]:
for i in range(len(shipper)):
    print(apply_ratio_vectorize(shipper.values,np.roll(shipper,i)))

In [ ]:
for i in range(1,5):
    print(apply_ratio_vectorize(shipper.values,np.roll(shipper,i)))

In [ ]:
#ratio('DHL GLOBAL FORWARDING LTD','DHL GLOBAL FORWARDING (NETHERLAND)')

In [ ]:
#ratio('DHL GLOBAL FORWARDING MALAYSIA SDN','DHL GLOBAL FORWARDING (NETHERLAND)')

In [ ]:
#ratio('DHL GLOBAL FORWARDING MALAYSIA SDN','DHL GLOBAL FORWARDING LTD')

In [ ]:
#ratio('DHL GLOBAL FORWARDING LTD','DHL GLOBAL FORWARDING (CANADA) INC')

In [ ]:
test = pd.DataFrame({'left':shipper.values,'right':np.roll(shipper,1),'score':apply_ratio_vectorize(shipper.values,np.roll(shipper,1))})

In [ ]:
test[test['score']>=0.75].sort_values('score').head()

In [23]:
import multiprocessing
from shipper_matching import multiprocess_apply_ratio
import itertools

#shift_stpes = [i for i in range(len(shipper))]
shift_stpes = [i for i in range(1,len(shipper))]
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    try:
        pool_outputs = pool.starmap(multiprocess_apply_ratio, list(zip(itertools.repeat(shipper),shift_stpes)))
    finally:
        pool.close()
        pool.join()

In [11]:
shift_stpes = [i for i in range(1,20)]

In [26]:
pd.concat(pool_outputs)

,left,right,score
3,YUTY INDUSTRIES CO LTD,SUMEEKO INDUSTRIES COLTD,0.782609
406,ASVINI FISHERIES PRIVATE LIMITED,ASVINI FISHERIES PVT LTD,0.857143
438,COMEL SPA,COSMA SPA,0.777778
463,THAI POLYACETAL CO LTD,THAI POLYCARBONATE CO LTD,0.765957
741,TTI MACAO COMMERCIAL OFFSHORE,TTI MACAO COMMERCIAL OFFSHORE LTD,0.906250
838,THE FISHIN COMPANY,THE FISHIN COMPANY,0.972973
1198,EVER-LOGISTICS INTERNATIONAL,UNIQUE LOGISTICS INTERNATIONAL (HK,0.774194
1257,PANALPINA WORLD TRANSPORT (I) PLTD,PANALPINA WORLD TRANSPORT (I)P LTD,0.970588
1418,MASTER INTERNATIONAL LOGISTICS,TRANS-POWER INTERNATONAL LOGISTICS,0.843750
1433,LOUIS DREYFUS COMPANY VIETNAM TRADI,LOUIS DREYFUS COMPANY VIETNAM TRA,0.970588
